![BERT](https://miro.medium.com/max/2960/0*63_xsVQp0Wezk9ua.jpg)

#  What is BERT?
* BERT stands for Bidirectional Encoders Representations from Transformers.
* Bidirectional - to understand the text you’re looking you’ll have to look back (at the previous words) and forward (at the next words)
* Transformers - The Attention Is All You Need paper presented the Transformer model. The Transformer reads entire sequences of tokens at once. In a sense, the model is non-directional, while LSTMs read sequentially (left-to-right or right-to-left). The attention mechanism allows for learning contextual relations between words (e.g. his in a sentence refers to Jim).
* (Pre-trained) contextualized word embeddings - The ELMO paper introduced a way to encode words based on their meaning/context. Nails has multiple meanings - fingernails and metal nails.

In this notebook we will be using transformers library by hugging face.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from tqdm.notebook import tqdm

Reading SMILE twitter emotion dataset

In [ ]:
df = pd.read_csv("/kaggle/input/smile-twitter-emotion-dataset/smile-annotations-final.csv",
                names = ['Id','Text','Category'])
df.set_index('Id',inplace = True)

df_new = pd.read_csv("../input/twitter-sentiment-analysis-hatred-speech/train.csv")
df_new_test = pd.read_csv("../input/twitter-sentiment-analysis-hatred-speech/test.csv")

df_new.set_index('id', inplace = True)
df[:4]

In [ ]:
df_new.shape

In [ ]:
df.Text.iloc[0]


* Counts of Emotions in the dataset

In [ ]:
# df.Category.value_counts()
df_new.label.value_counts()


We will be filtering the emotions with single emotions and ignoring the rest.

In [ ]:
df = df[(df.Category!="nocode")]
df = df[~(df.Category.str.contains("\|"))]
df.Category.value_counts()

In [ ]:
# possible_label = df.Category.unique()
dict_label = {'happy': 0,
             'angry': 1}
# for index,possible_label in enumerate(possible_label):
#     dict_label[possible_label] = index
dict_label

In [ ]:
df["Label"] = df["Category"].replace(dict_label)
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (5,5)
sns.countplot(df["Label"],hue = df["Label"],palette = 'dark')
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
labels = [0,1,2,3,4,5]
sizes = df["Label"].value_counts()
colors = plt.cm.copper(np.linspace(0, 1, 5))
explode = [0.1, 0.1,0.1, 0.2, 0.5, 0.9]
cmap = plt.get_cmap('Spectral')
plt.pie(sizes,labels = labels,colors = colors,shadow = True,explode = explode)
plt.legend()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test = train_test_split(df.index.values,
#                                                 df.Label.values,
#                                                 test_size = 0.15,
#                                                 random_state=17,
#                                                 stratify = df.Label.values)

# X_train = df_new_train.tweet
# X_test = df_new_test.tweet

# y_train = df_new_train.label
# y_


X_train,X_test,y_train,y_test = train_test_split(df_new.index.values,
                                                df_new.label.values,
                                                test_size = 0.15,
                                                random_state=17,
                                                stratify = df_new.label.values)


In [ ]:
y_test

In [ ]:
y_train

In [ ]:
X_train

In [ ]:
# df["data_type "] = ['not_set']*df.shape[0]
# df.head()

In [ ]:
# for i in df.index:
#     if i in X_train:
#         df["data_type"].replace("not_set","train")
#     elif i in X_test:
#         df["data_type"].replace("not_set","test")
# df.head() 
df_new.loc[X_train,'data_type'] = 'train'
df_new.loc[X_test,'data_type'] = 'test'

In [ ]:
df_new.head()

In [ ]:
df_new.groupby(['label','data_type']).count()

# Modeling

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case = True)

Encoding text by tokenizing using BERT Tokenizer
* In order to use BERT text embeddings as input to train text classification model, we need to tokenize our text reviews. Tokenization refers to dividing a sentence into individual words. To tokenize our text, we will be using the BERT tokenizer

In [ ]:
encoder_train = tokenizer.batch_encode_plus(df_new[df_new["data_type"]=='train'].tweet.values,
                                           add_special_tokens = True,
                                            return_attention_masks = True,
                                           pad_to_max_length = True,
                                           max_length = 256,
                                           return_tensors = 'pt')



encoder_test = tokenizer.batch_encode_plus(df_new[df_new["data_type"]=='test'].tweet.values,
                                           add_special_tokens = True,
                                            return_attention_masks = True,
                                           pad_to_max_length = True,
                                           max_length = 256,
                                           return_tensors = 'pt')

input_ids_train = encoder_train['input_ids']
attention_masks_train = encoder_train["attention_mask"]
labels_train = torch.tensor(df_new[df_new['data_type']=='train'].label.values)


input_ids_test = encoder_test['input_ids']
attention_masks_test = encoder_test["attention_mask"]
labels_test = torch.tensor(df_new[df_new['data_type']=='test'].label.values)

In [ ]:
data_train = TensorDataset(input_ids_train,attention_masks_train,labels_train)
data_test = TensorDataset(input_ids_test,attention_masks_test,labels_test)


In [ ]:
len(data_train),len(data_test)

* We will use sequence classification model as we have to classify multi label text from the dataset.

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                     num_labels = len(dict_label),
                                     output_attentions = False,
                                     output_hidden_states =  False)

From torch we will use data loader,randomsampler to load data in an iterable format but extracting different subsamples from dataset.

In [ ]:
from torch.utils.data import RandomSampler,SequentialSampler,DataLoader

dataloader_train = DataLoader(
    data_train,
    sampler= RandomSampler(data_train),
    batch_size = 16
    
)


dataloader_test = DataLoader(
    data_test,
    sampler= RandomSampler(data_test),
    batch_size = 32
    
)

In [ ]:
from transformers import AdamW,get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(),lr = 1e-5,eps = 1e-8)

epochs  = 1
scheduler = get_linear_schedule_with_warmup(
            optimizer,
    num_warmup_steps = 0,
   num_training_steps = len(dataloader_train)*epochs 
)

Defining Model metrics

In [ ]:
from sklearn.metrics import f1_score 

def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,preds_flat,average = 'weighted')


In [ ]:
def accuracy_per_class(preds,labels):
    label_dict_reverse = {v:k for k,v in dict_label.items()}
    
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f"Class:{label_dict_reverse}")
        print(f"Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n")

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"Loading:{device}")

Defining Evaluation

In [ ]:
def evaluate(dataloader_val):
    model.eval()
    
    loss_val_total = 0
    predictions,true_vals = [],[]
    
    for batch in tqdm(dataloader_val):
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':  batch[0],
                  'attention_mask':batch[1],
                  'labels': batch[2]
                 }
        with torch.no_grad():
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total +=loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
        
        
    loss_val_avg = loss_val_total/len(dataloader_val)  
    
    predictions = np.concatenate(predictions,axis=0)
    true_vals = np.concatenate(true_vals,axis=0) 
    return loss_val_avg,predictions,true_vals
            

* Training Data

In [ ]:
for epoch in tqdm(range(1,epochs+1)):
    model.train()
    
    loss_train_total=0
    
    progress_bar = tqdm(dataloader_train,desc = "Epoch: {:1d}".format(epoch),leave = False,disable = False)
    
    
    for batch in progress_bar:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            "input_ids":batch[0],
            "attention_mask":batch[1],
            "labels":batch[2]
            
        }
        outputs = model(**inputs)
        
        loss = outputs[0]
#         logits = outputs[1]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm(model.parameters(),1.0)
        
        optimizer.step()
        scheduler.step()
        
        
        progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})
#     torch.save(model.state_dict(),f'/kaggle/output/BERT_ft_epoch{epoch}.model')To save the model after each epoch
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training Loss: {loss_train_avg}')
    val_loss,predictions,true_vals = evaluate(dataloader_test)
    test_score = f1_score_func(predictions,true_vals)
    tqdm.write(f'Val Loss:{val_loss}\n Test Score:{test_score}')
    

# Using the saved model

In [ ]:
# from transformers import BertForSequenceClassification
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
#                                      num_labels = len(dict_label),
#                                      output_attentions = False,
#                                      output_hidden_states =  False)

In [ ]:
model.to(device)


In [ ]:
# #using saved model
# model.load_state_dict(torch.load("Path of saved model"))# in case want to use the saved model
# _,predictions,true_vals = evaluate(dataloader_test)
# accuracy_per_class(predictions,true_vals)

In [ ]:
_,predictions,true_vals = evaluate(dataloader_test)

In [ ]:
accuracy_per_class(predictions,true_vals)

In [ ]:
print('On a '+str(4407/4459)+'% de juste pour les tweets bienveillants et '+str(232/336)+' pour les tweets hostiles')

In [ ]:
print('On a {}% de juste pour les tweets bienveillants et {}% pour les tweets hostiles'.format(440700//4459, 23200//336))